# Music Genre and Composer Classification Using Deep Learning

## Classical Music Time Series Model Comparison
## AAI-511 Team 6 Final Project
## Part I : Data Preprocessing & Feature Extraction

Team 6:  Philip Felizarta, Sarah Durrani, Geoffrey Fadera

University of San Diego, Applied Artificial Intelligence

Date:  August 12, 2024

GitHub Repository: https://github.com/PhilipFelizarta/Classical-Music-Time-Series-Model-Comparison?query=PhilipFelizarta

In [2]:
import os
import pretty_midi
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import zipfile
import pandas as pd
import warnings
from sklearn.preprocessing import LabelBinarizer

RANDOM_STATE = 0

## Step 0. Unzip Data and all subdirectories

In [2]:
# By visual inspection, even after unzipping the parent directory 'archive', composers' subdirectories still contain Zip files.
# The following function unzips a Zip file, and recursively unzips all Zip files within it.

def unzipFileTree(directory, file):
    
    if file.endswith('.zip'):
        zip_filepath = os.path.join(directory, file)
        zip_filename = os.path.splitext(os.path.basename(zip_filepath))[0]
        zip_targetDir = os.path.join(directory, zip_filename)
        
            
        try:
            with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
                zip_ref.extractall(zip_targetDir)
                #print(f"\tUnzipped File: {zip_filepath}")
                #print(f"\tUnzipped to folder: {zip_targetDir}")
        except zipfile.BadZipFile:
            print(f"Error: {zip_filepath} is a bad zip file")
    else:
        return

    for root, dirs, files in os.walk(zip_targetDir):
        for file in files:
            unzipFileTree(root, file)
            
            

In [3]:
# Step 0. Place 'archive.zip' in the same folder as this ipynb file and unzip using unzipFileTree () function.
unzipFileTree('./', 'archive.zip')

## Step 1. List all midi files available for each composer

In [4]:
# List filepaths for all midi files within the composer folder.

def list_midi_filepaths(directory, composer = 'Alkan'):
    
    composerDir = os.path.join(directory, composer)
    midi_count = 0
    non_midi_count = 0
    filepaths = []
    #print()
    
    for dirpath, dirs, files in os.walk(composerDir):  
        for filename in files: 
            fname = os.path.join(dirpath,filename) 
            if (fname.endswith('.mid') or fname.endswith('.MID')): 
                filepaths.append(fname)
                midi_count = midi_count + 1
                #print(fname)
            else :
                non_midi_count = non_midi_count + 1
                print(fname)
    
    return midi_count, filepaths
    

In [5]:
# Step 1. Load Beethoven midi fliles
root = "./archive/midiclassics"
cntBach, midisPathBach = list_midi_filepaths(root, 'Bach')
cntBeet, midisPathBeet = list_midi_filepaths(root, 'Beethoven')
cntChop, midisPathChop = list_midi_filepaths(root, 'Chopin')
cntMoza, midisPathMoza = list_midi_filepaths(root, 'Mozart')

./archive/midiclassics/Bach/Bwv001- 400 Chorales/readme
./archive/midiclassics/Beethoven/Complete Tempest Sonata.zip
./archive/midiclassics/Beethoven/Complete Sonata in A flat major Op.110.zip
./archive/midiclassics/Beethoven/Sonata No.1 in D.zip
./archive/midiclassics/Beethoven/Piano Sonata No.18.zip
./archive/midiclassics/Beethoven/Piano and Cello Sonata No.2 Op 5.zip
./archive/midiclassics/Beethoven/Piano Sonata No.23.zip


In [6]:
# Check Data Distribution
cntAll = pd.DataFrame({'Count': [cntBach, cntBeet, cntChop, cntMoza]}, 
                      index = ['Bach', 'Beethoven', 'Chopin', 'Mozart'])
cntAll

,Count
Bach,1024
Beethoven,220
Chopin,136
Mozart,257


### NOTE: There is clear class imbalance with the 'Bach' label having at least 4 times as many samples as the rest of the classes.

## Step 2. Data Preprocessing

### Step 2.1. Read midi files using PrettyMidi and remove invalid entries

In [7]:
# Read midi files using prettyMidi

def read_Midis(midisPath = []):
    
    midis_data = []
    for file in midisPath:
        
        try:
           # with warnings.catch_warnings():
           #     warnings.simplefilter("error")
            pm = pretty_midi.PrettyMIDI(file)
            
        except:
            print(f"Error loading MIDI file:", file)
            continue
        # Handle the error as needed, for example, log it or skip the file
        
        midis_data = np.append(midis_data,pm)
       
        
        
    return midis_data

In [8]:
# read each midi file and store successfully parsed midi data into variables.

midisBach = []
midisBeet = []
midisChop = []
midisMoza = []

midisBach = read_Midis(midisPathBach)
midisBeet = read_Midis(midisPathBeet)
midisChop = read_Midis(midisPathChop)
midisMoza = read_Midis(midisPathMoza)


/Users/geoffreyfadera/anaconda3/lib/python3.11/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


Error loading MIDI file: ./archive/midiclassics/Beethoven/Anhang 14-3.mid
Error loading MIDI file: ./archive/midiclassics/Mozart/Piano Sonatas/Nueva carpeta/K281 Piano Sonata n03 3mov.mid


In [9]:
# Check new data distribution after removing invalid midi files

cntMidis = pd.DataFrame({'Count': [len(midisBach), len(midisBeet), len(midisChop), len(midisMoza)]}, 
                      index = ['Bach', 'Beethoven', 'Chopin', 'Mozart'])
cntMidis

,Count
Bach,1024
Beethoven,219
Chopin,136
Mozart,256


### Step 2.2. Combine all prettyMidi data into one array, and create an array containing corresponding labels

In [10]:
midisAll = []
midisAll = np.append(midisAll, midisBach)
midisAll = np.append(midisAll, midisBeet)
midisAll = np.append(midisAll, midisChop)
midisAll = np.append(midisAll, midisMoza)

labelsAll = []
labelsAll = np.append(labelsAll, np.full(len(midisBach), 'Bach'))
labelsAll = np.append(labelsAll, np.full(len(midisBeet), 'Beet'))
labelsAll = np.append(labelsAll, np.full(len(midisChop), 'Chop'))
labelsAll = np.append(labelsAll, np.full(len(midisMoza), 'Moza'))


In [11]:
midisAll.shape

(1635,)

In [12]:
labelsAll.shape

(1635,)

### Step 2.3. Convert Labels to Binary Representations

In [13]:
lb_enc = []
labelsAllDense = []

lb_enc = LabelBinarizer()
labelsAllDense = lb_enc.fit_transform(labelsAll)
labelsAllDense.shape


(1635, 4)

### Step 2.4 Split into Train, Validation, and Test Sets

In [14]:
TRAIN_SIZE = 0.80
VAL_TEST_SPLIT = 0.5

X_train, X_valtest, y_train, y_valtest = train_test_split(midisAll, labelsAllDense, test_size=1 - TRAIN_SIZE, random_state=RANDOM_STATE)
X_val, X_test, y_val, y_test = train_test_split(X_valtest, y_valtest, test_size=VAL_TEST_SPLIT, random_state=RANDOM_STATE)

# Save y train, val and test values to a directory.
# These values are common to all data_preprocessing and feature_engineering variations

np.save('./y_train.npy', y_train)
np.save('./y_val.npy', y_val)
np.save('./y_test.npy', y_test)

### Step 2.5 Convert PrettyMidi data into formats suitable for deep learning models. The following format options are explored in this project

### STEP 2.5.1 (Option 1): Convert each midi file as a BAG of NOTES,  with each note entry containing {start, end, pitch, velocity, and instrument program index} values. 

**prettyMidi vector :** num_instruments X num_notes_per_inst X ([ start_time, end_time, pitch, velocity])<br>
**flattened vector :** total_num_notes X [start_time, end_time, pitch, velocity, InstrumentProgramIndex]


In [15]:
# define a function that converts a pretty_midi array into a 'bag of notes'
# Dimensionality is reduced as Time and Instrument components are now added as features for each note.

def convertMidi_OP1 (midiData = []):
    
    flatMidi = np.empty((0,5), float)
    start = []
    end = []
    pitch = []
    velocity = []
    i = 0
    
    for instrument in midiData.instruments:
        for note in instrument.notes:
            start = note.start
            end = note.end  
            pitch = note.pitch
            velocity = note.velocity

            combinedEntry = np.array([start, end, pitch, velocity, instrument.program]).reshape(1,-1)
            flatMidi = np.append(flatMidi,combinedEntry , axis = 0)
            
    return flatMidi

In [16]:
# Get Max Number of Unique Notes

def getMaxNumNotesMidiList_OP1 (midiDataList = []):
    
    numNotes = 0
    i = 0
    
    for midi in midiDataList:
        #print("midi entry i:", i)
        temp = len(convertMidi_OP1(midi))
        #print("\tlen:", temp)
        if (temp > numNotes):
            #print("\tNumNotes Updated at entry ", i)
            #print("\tlen:", temp)
            numNotes = temp
        
        i = i+1
    
    return numNotes
   
    
maxNotes = getMaxNumNotesMidiList_OP1(midisAll)
print("Max Note Entries: ", maxNotes)


Max Note Entries:  47745


In [17]:
# Convert each midi file into a collection of note entries each of size(maxNotes,5)
# if the number of note entries for a particular file is less than maxNotes, pad zeros
# save new vector as a new file.

def normalizeMidiSize_OP1(midiDataList = [], maxEntries = [], fileDir = []):
    #midis_result = np.empty((0,maxEntries,5), float)
    i = 0
    
    # create directories to store normalized data
    if not os.path.exists(fileDir):
        os.makedirs(fileDir)
        print(f"Directory '{fileDir}' created successfully")
    else:
        print(f"Directory '{fileDir}' already exists")
    
    for midi in midiDataList:
        
        if (i%20==0): 
            print("Update: Entry no ", i)
            #break
        
        temp = convertMidi_OP1(midi)
        #print("\tmidi (temp) new shape: ", temp.shape)
        tempZeros = np.zeros(((maxEntries-len(temp)),5))
        #print("\tmidi (tempZeros) shape: ", tempZeros.shape)
        temp2 = np.append(temp, tempZeros, axis =0).reshape(maxEntries,5)
        #print("\tmidi (temp2) new shape: ", temp2.shape)
        
        filename = 'X_'+str(i)+'.npy'
        filePath_ = os.path.join(fileDir,filename) 
        
        np.save(filePath_, temp2)
        #midis_result = np.append(midis_result, temp2, axis = 0)
        #print("\tmidi (result) new shape: ", midis_result.shape)
        
        i = i+1
    return 
    

In [18]:
normalizeMidiSize_OP1(X_train, maxNotes, fileDir = './OP1/Train')
normalizeMidiSize_OP1(X_val, maxNotes, fileDir = './OP1/Val')
normalizeMidiSize_OP1(X_test, maxNotes, fileDir = './OP1/Test')

Directory './OP1/Train' created successfully
Update: Entry no  0
Update: Entry no  20
Update: Entry no  40
Update: Entry no  60
Update: Entry no  80
Update: Entry no  100
Update: Entry no  120
Update: Entry no  140
Update: Entry no  160
Update: Entry no  180
Update: Entry no  200
Update: Entry no  220
Update: Entry no  240
Update: Entry no  260
Update: Entry no  280
Update: Entry no  300
Update: Entry no  320
Update: Entry no  340
Update: Entry no  360
Update: Entry no  380
Update: Entry no  400
Update: Entry no  420
Update: Entry no  440
Update: Entry no  460
Update: Entry no  480
Update: Entry no  500
Update: Entry no  520
Update: Entry no  540
Update: Entry no  560
Update: Entry no  580
Update: Entry no  600
Update: Entry no  620
Update: Entry no  640
Update: Entry no  660
Update: Entry no  680
Update: Entry no  700
Update: Entry no  720
Update: Entry no  740
Update: Entry no  760
Update: Entry no  780
Update: Entry no  800
Update: Entry no  820
Update: Entry no  840
Update: Entry n

In [19]:
X_train_OP1 = np.load('./OP1/Train/X_0.npy')

In [21]:
X_train_OP1[0:5]

array([[ 1.875,  2.5  , 72.   , 96.   ,  0.   ],
       [ 2.5  ,  3.125, 69.   , 96.   ,  0.   ],
       [ 3.125,  3.75 , 72.   , 96.   ,  0.   ],
       [ 3.75 ,  4.375, 74.   , 96.   ,  0.   ],
       [ 4.375,  5.   , 72.   , 96.   ,  0.   ]])

In [4]:
X_val_OP1 = np.load('./OP1/Val/X_162.npy')
X_val_OP1[0:5]

array([[0.00000000e+00, 6.81818750e-02, 6.90000000e+01, 6.40000000e+01,
        0.00000000e+00],
       [6.81818750e-02, 1.36363750e-01, 6.80000000e+01, 5.60000000e+01,
        0.00000000e+00],
       [1.36363750e-01, 5.22727708e-01, 6.90000000e+01, 6.40000000e+01,
        0.00000000e+00],
       [5.45455000e-01, 8.18182500e-01, 6.90000000e+01, 6.40000000e+01,
        0.00000000e+00],
       [8.18182500e-01, 8.86364375e-01, 6.90000000e+01, 6.40000000e+01,
        0.00000000e+00]])

**2.5.1 SUMMARY:** Each Midi file is converted to a 47745x5 array and the values are saved to a new file. 
Later, these new files are reloaded (see jupyter notebook part 2) for further processing/modelling.

### STEP 2.5.2: Convert all Midi Files to be of (instrument_idx, pitch, velocity,  time)
#### Each Midi file will be flattened as in Option 1.
#### Flattened array will be sampled based on desired timing resolution.
#### Pitch values are ( 0,127) - 7 bits
#### velocity values are (0,127) - 7 bits
#### instrument_idx are (0,127) - 7 bits

get instruments values - encode using oneHotEncoding, instrument index should be made independent of each other.
get pitch values - ordinal encoding, the higher the pitch, the higher the frequency
get velocity values - ordinal encoding, the higher the velocity, the higher the intensity of the pitch being played


In [393]:
instruments_idx = midisAll_OP1[:,:,4]
valuesI, countsI = np.unique(instruments_idx, return_counts = True)
len(valuesI)

62

In [394]:
pitches = midisAll_OP1[:,:,2]
valuesP, countsP = np.unique(pitches, return_counts = True)
len(valuesP)

96

In [395]:
velocities = midisAll_OP1[:,:,3]
valuesV, countsV = np.unique(velocities, return_counts = True)
len(valuesV)

128

In [403]:
countsV

array([72900064,      185,       22,       12,       10,       18,
             21,       23,       38,       31,      373,       71,
             81,      249,      577,      591,      314,      488,
           1272,      720,     1576,     1320,     4328,     2992,
           3565,     3798,     3950,     4139,     5661,     6231,
          22536,     7685,    16361,    10455,    12104,    13142,
          13908,    11918,    14770,    14551,    39282,    16227,
          19356,    23299,    33882,    99601,    25186,    28159,
          58698,    39619,    68250,    33511,    34847,    31807,
          29086,    32678,    35192,    30529,    75535,    26385,
          90719,    25050,    63260,    33089,   333054,    26158,
          36145,    22691,    41171,    22740,    79398,    34670,
         133016,    26712,    39727,    66923,   187987,    23022,
          36605,    21660,   188000,    22361,    41699,    15824,
          27386,    53435,    25819,    23297,    58847,    40

In [339]:
midisAll_OP1 = convertReshapeMidiList_OP1(midisAll, maxNotes)
midisAll_OP1.shape

print("midisAll_OP1 shape: ", midisAll_OP1.shape)

Entry no  0
Entry no  1
Entry no  2
Entry no  3
Entry no  4
Entry no  5
Entry no  6
Entry no  7
Entry no  8
Entry no  9
Entry no  10
Entry no  11
Entry no  12
Entry no  13
Entry no  14
Entry no  15
Entry no  16
Entry no  17
Entry no  18
Entry no  19
Entry no  20
Entry no  21
Entry no  22
Entry no  23
Entry no  24
Entry no  25
Entry no  26
Entry no  27
Entry no  28
Entry no  29
Entry no  30
Entry no  31
Entry no  32
Entry no  33
Entry no  34
Entry no  35
Entry no  36
Entry no  37
Entry no  38
Entry no  39
Entry no  40
Entry no  41
Entry no  42
Entry no  43
Entry no  44
Entry no  45
Entry no  46
Entry no  47
Entry no  48
Entry no  49
Entry no  50
Entry no  51
Entry no  52
Entry no  53
Entry no  54
Entry no  55
Entry no  56
Entry no  57
Entry no  58
Entry no  59
Entry no  60
Entry no  61
Entry no  62
Entry no  63
Entry no  64
Entry no  65
Entry no  66
Entry no  67
Entry no  68
Entry no  69
Entry no  70
Entry no  71
Entry no  72
Entry no  73
Entry no  74
Entry no  75
Entry no  76
Entry no 

Entry no  594
Entry no  595
Entry no  596
Entry no  597
Entry no  598
Entry no  599
Entry no  600
Entry no  601
Entry no  602
Entry no  603
Entry no  604
Entry no  605
Entry no  606
Entry no  607
Entry no  608
Entry no  609
Entry no  610
Entry no  611
Entry no  612
Entry no  613
Entry no  614
Entry no  615
Entry no  616
Entry no  617
Entry no  618
Entry no  619
Entry no  620
Entry no  621
Entry no  622
Entry no  623
Entry no  624
Entry no  625
Entry no  626
Entry no  627
Entry no  628
Entry no  629
Entry no  630
Entry no  631
Entry no  632
Entry no  633
Entry no  634
Entry no  635
Entry no  636
Entry no  637
Entry no  638
Entry no  639
Entry no  640
Entry no  641
Entry no  642
Entry no  643
Entry no  644
Entry no  645
Entry no  646
Entry no  647
Entry no  648
Entry no  649
Entry no  650
Entry no  651
Entry no  652
Entry no  653
Entry no  654
Entry no  655
Entry no  656
Entry no  657
Entry no  658
Entry no  659
Entry no  660
Entry no  661
Entry no  662
Entry no  663
Entry no  664
Entry 

KeyboardInterrupt: 

In [335]:
midisAll_OP1[1][3156]

array([398.795695, 412.770389,  43.      , 102.      ,   0.      ])

In [282]:
midisAll_OP1 = np.empty((1,1,5), float)
print("\tmidisAll_OP1 shape: ", midisAll_OP1.shape)
i = 0

for midi in midisAll:
    print("Entry no ", i)
    if (i==3): break
    temp = convertMidi_OP1(midi).reshape(1,-1,5)
    print("\tmidi (temp) new shape: ", temp.shape)
    #midisAll_OP1 = np.append(midisAll_OP1, temp, axis=1)
   
    midisAll_OP1 = np.concatenate((midisAll_OP1, temp), axis=1)
    #midisAll_OP1 = np.vstack((midisAll_OP1, temp))
    i = i+1

	midisAll_OP1 shape:  (1, 1, 5)
Entry no  0
	midi (temp) new shape:  (1, 977, 5)
Entry no  1
	midi (temp) new shape:  (1, 3157, 5)
Entry no  2
	midi (temp) new shape:  (1, 1050, 5)
Entry no  3


In [279]:
midisAll_OP1.shape

(1, 5185, 5)

In [258]:
midisAll_OP1[0].shape

(5185, 5)

In [185]:
len(midisAll[0].instruments[0].notes)

741

In [186]:
len(midisAll[0].instruments[1].notes)

236

In [192]:
temp = convertMidi_OP1(midisAll[144])

In [193]:
temp.shape

(758, 5)

In [196]:
midisAll_OP1[144]

24.0

In [114]:
len(midi_data3.instruments[1].notes)

1985

In [116]:
midi_data3.instruments[0].notes[0:5]

[Note(start=0.000000, end=0.273438, pitch=72, velocity=60),
 Note(start=0.000000, end=0.367188, pitch=65, velocity=60),
 Note(start=0.281250, end=0.367188, pitch=74, velocity=60),
 Note(start=0.375000, end=0.742188, pitch=72, velocity=60),
 Note(start=0.375000, end=1.000000, pitch=67, velocity=60)]

In [117]:
midi_data3.instruments[1].notes[0:5]

[Note(start=0.000000, end=0.367188, pitch=57, velocity=60),
 Note(start=0.000000, end=0.367188, pitch=60, velocity=60),
 Note(start=0.375000, end=1.000000, pitch=52, velocity=60),
 Note(start=0.375000, end=1.000000, pitch=60, velocity=60),
 Note(start=1.125000, end=1.492188, pitch=53, velocity=60)]

In [63]:
features3[0]

[53,
 57,
 60,
 65,
 69,
 60,
 65,
 69,
 53,
 57,
 60,
 65,
 69,
 60,
 65,
 69,
 53,
 55,
 62,
 67,
 70,
 62,
 67,
 70,
 53,
 55,
 62,
 67,
 70,
 62,
 67,
 70,
 52,
 55,
 60,
 67,
 70,
 60,
 67,
 70,
 52,
 55,
 60,
 67,
 70,
 60,
 67,
 70,
 53,
 57,
 60,
 65,
 69,
 60,
 65,
 69,
 53,
 57,
 60,
 65,
 69,
 60,
 65,
 69,
 53,
 57,
 60,
 65,
 53,
 57,
 60,
 65,
 53,
 57,
 60,
 65,
 53,
 57,
 60,
 65,
 53,
 55,
 58,
 62,
 53,
 55,
 58,
 62,
 53,
 55,
 58,
 62,
 53,
 55,
 58,
 62,
 52,
 55,
 58,
 60,
 52,
 55,
 58,
 60,
 52,
 55,
 58,
 60,
 52,
 55,
 58,
 60,
 53,
 57,
 60,
 65,
 53,
 57,
 60,
 65,
 53,
 57,
 60,
 60,
 65,
 53,
 57,
 60,
 65,
 53,
 57,
 62,
 65,
 53,
 57,
 62,
 65,
 53,
 57,
 62,
 65,
 53,
 57,
 62,
 65,
 53,
 55,
 59,
 62,
 53,
 55,
 59,
 62,
 53,
 55,
 59,
 62,
 53,
 55,
 59,
 62,
 52,
 55,
 60,
 64,
 52,
 55,
 60,
 64,
 52,
 55,
 60,
 64,
 52,
 55,
 60,
 64,
 52,
 53,
 57,
 60,
 52,
 53,
 57,
 60,
 52,
 53,
 57,
 60,
 52,
 53,
 57,
 60,
 50,
 53,
 57,
 60,
 50,
 53,
 57,


In [66]:
features3[0][643:647]

[53, 65, 41, 41]

In [118]:
# Example # 4 Manual
# Load dataset
root = "./archive/midiclassics/Beethoven"
file4 = "Violin Concerto op61 2-3movs.mid"
midi_file4 = os.path.join(root, file4)
midi_data4 = pretty_midi.PrettyMIDI(midi_file4, resolution = 100)

In [119]:
midi_data4.instruments

[Instrument(program=73, is_drum=False, name="Flauti I II"),
 Instrument(program=68, is_drum=False, name="Oboi I II"),
 Instrument(program=71, is_drum=False, name="Clarinetti in C I II"),
 Instrument(program=71, is_drum=False, name="Clarinetti in A I II"),
 Instrument(program=70, is_drum=False, name="Fagotti I II"),
 Instrument(program=60, is_drum=False, name="Corni in G I II"),
 Instrument(program=60, is_drum=False, name="Corni in D I II"),
 Instrument(program=56, is_drum=False, name="Trombe in D I II"),
 Instrument(program=47, is_drum=False, name="Timpani"),
 Instrument(program=41, is_drum=False, name="Violino solo"),
 Instrument(program=46, is_drum=False, name="Violino solo"),
 Instrument(program=50, is_drum=False, name="Violino I"),
 Instrument(program=45, is_drum=False, name="Violino I"),
 Instrument(program=48, is_drum=False, name="Violino I"),
 Instrument(program=48, is_drum=False, name="Violino II"),
 Instrument(program=45, is_drum=False, name="Violino II"),
 Instrument(program=

In [15]:
# Load dataset
root = "./archive/midiclassics/Bach"
#file = "AveMaria.mid"

#print(os.path.join(root, file))

#midi_data = pretty_midi.PrettyMIDI(os.path.join(root, file))

midi_files = [os.path.join(root, file) for root, _, files in os.walk('path_to_midi_files')
#                                          for file in files if file.endswith('.mid')]

# Extract features and labels
features = [extract_features(file) for file in midi_files]
labels = [0 if 'genre1' in file else 1 for file in midi_files]  # Example labels


./archive/midiclassics/Bach/AveMaria.mid


In [16]:
np.array(midi_data).shape

()

In [17]:
midi_data.instruments

[Instrument(program=0, is_drum=False, name=""),
 Instrument(program=0, is_drum=False, name=""),
 Instrument(program=0, is_drum=False, name="")]

In [20]:
midi_data.instruments[0]

Instrument(program=0, is_drum=False, name="")

In [ ]:



# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)

# Train an SVM classifier
svm = SVC(kernel='rbf')
svm.fit(X_train, y_train)

# Evaluate the model
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')